In [44]:
from time import time
from typing import Optional, Sequence
import pandas as pd
import numpy as np

In [45]:
def get_flow_value(cur_solution: list, dist: list, flow: list):
    value = 0
    for i in range(len(cur_solution) - 1):
        for j in range(i + 1, len(cur_solution)):
            value += dist[i][j] * flow[cur_solution[i]][cur_solution[j]]
    value *= 2
    return value

In [46]:
def swap_factories(solution: list, i: int, j: int):
    sol_copy = solution.copy()
    tmp = sol_copy[i]
    sol_copy[i] = sol_copy[j]
    sol_copy[j] = tmp
    return sol_copy

In [47]:
def local_search(n_factories: int, dist: list, flow: list, starting_point: Optional[Sequence[int]] = None):
    '''Local search with first-improvement + don't look bits'''
    start = time()
    if starting_point is None:
        best_sol = [i for i in range(n_factories)]
    else:
        best_sol = starting_point
    best_sol_value = get_flow_value(best_sol, dist, flow)
    while True:
        dlb = [False for i in range(n_factories)]
        improvement_flag = False
        for i in range(n_factories):
            dlb_flag = True
            for j in range(n_factories):
                if not dlb[j] and i != j:
                    new_sol = swap_factories(best_sol, i, j)
                    delta = 0
                    for k in range(n_factories):
                        if k != i and k != j:
                            delta -= dist[i][k] * \
                                flow[best_sol[i]][best_sol[k]]
                            delta -= dist[j][k] * \
                                flow[best_sol[j]][best_sol[k]]
                            delta += dist[i][k] * flow[new_sol[i]][new_sol[k]]
                            delta += dist[j][k] * flow[new_sol[j]][new_sol[k]]
                    delta *= 2
                    if delta < 0:
                        dlb_flag = False
                        improvement_flag = True
                        best_sol_value += delta
                        best_sol = new_sol
            if dlb_flag:
                dlb[i] = True
        if not improvement_flag:
            break

        # print(best_sol, best_sol_value, '\n')
    end = time()

    return best_sol, best_sol_value, end - start

In [48]:
data = []

In [49]:
files_list = ['tai20a.txt', 'tai40a.txt', 'tai60a.txt', 'tai80a.txt', 'tai100a.txt']
 
for file in files_list:       
    with open(f'benchmarks/{file}', 'r') as f:
        dist = []
        flow = []
        n_factories = int(f.readline())
        for i in range(n_factories):
            arr = f.readline().split()
            arr = list(map(int, arr))
            dist.append(arr)
        
        f.readline()
        for i in range(n_factories):
            arr = f.readline().split()
            arr = list(map(int, arr))
            flow.append(arr)
        
        best_sol, best_sol_value, dt = local_search(n_factories, dist, flow)
        data.append([file[:-4] , best_sol_value, dt])
        print(f'Best solution: {best_sol}')
        print(f'Best solution flow value: {best_sol_value}')
        print(f'Time: {dt}\n')
        fsave = file.replace('txt','sol')
        with open(f'ls_outputs/{fsave}', 'w') as output_file:
            print(*best_sol, file=output_file)

Best solution: [16, 3, 17, 14, 7, 19, 10, 18, 1, 9, 13, 11, 12, 6, 15, 0, 2, 8, 4, 5]
Best solution flow value: 736450
Time: 0.011997461318969727

Best solution: [23, 28, 39, 22, 15, 37, 34, 13, 0, 9, 3, 26, 25, 24, 5, 27, 21, 17, 6, 8, 20, 31, 1, 19, 12, 16, 35, 10, 2, 29, 14, 38, 32, 30, 33, 18, 36, 7, 11, 4]
Best solution flow value: 3260316
Time: 0.15450096130371094

Best solution: [52, 35, 21, 6, 10, 32, 56, 7, 23, 11, 55, 27, 4, 36, 37, 24, 44, 17, 18, 43, 9, 47, 48, 0, 16, 49, 50, 26, 20, 28, 15, 31, 8, 33, 2, 39, 13, 29, 41, 25, 40, 14, 1, 38, 42, 30, 45, 22, 3, 19, 46, 51, 59, 53, 12, 54, 5, 57, 58, 34]
Best solution flow value: 7467702
Time: 0.5945019721984863

Best solution: [20, 39, 3, 16, 11, 12, 26, 7, 24, 36, 8, 51, 40, 47, 65, 10, 74, 38, 18, 17, 19, 9, 33, 23, 77, 78, 14, 21, 54, 29, 6, 30, 68, 60, 13, 2, 79, 75, 0, 31, 22, 43, 34, 5, 27, 35, 45, 4, 48, 41, 1, 37, 50, 53, 15, 49, 52, 62, 57, 25, 32, 61, 58, 44, 64, 59, 63, 67, 46, 66, 69, 71, 70, 73, 55, 72, 76, 56, 28

In [50]:
ls_df = pd.DataFrame(data, columns=['sample', 'LS best value', 'LS time'])
ls_df

,sample,LS best value,LS time
0,tai20a,736450,0.011997
1,tai40a,3260316,0.154501
2,tai60a,7467702,0.594502
3,tai80a,14061060,1.082001
4,tai100a,21771672,1.810001


**Iterated Local Search**

In [51]:
data=[]

In [52]:
def inverse_k_perturbator(in_seq: Optional[Sequence[int]] = None, k: Optional[int]=None):
    if in_seq is None:
        return None
    else: 
        l = len(in_seq)
        if k is None:
            p1, p2 = sorted([np.random.randint(0, l), np.random.randint(0, l)])
        else:
            p1 = np.random.randint(0, l)
            p2 = p1 + k % l
            p1,p2 = sorted([p1,p2])
        return in_seq[:p1] + list(reversed(in_seq[p1:p2])) + in_seq[p2:]


In [53]:
def ils(n_factories: int, dist: list, flow: list, 
         perturbation, n_iters = 1, starting_point: Optional[Sequence[int]] = None):
    times = []
    best_sol = starting_point
    best_sol_value = np.inf
    for i in range(n_iters):
        new_staring_point = perturbation(best_sol)
        new_sol, new_sol_value, dt = local_search(n_factories, dist, flow, new_staring_point)
        times.append(dt)
        if new_sol_value < best_sol_value:
            best_sol_value = new_sol_value
            best_sol = new_sol

    return best_sol, best_sol_value, sum(times)

In [54]:
files_list = ['tai20a.txt', 'tai40a.txt', 'tai60a.txt', 'tai80a.txt', 'tai100a.txt']
 
for file in files_list:       
    with open(f'benchmarks/{file}', 'r') as f:
        dist = []
        flow = []
        n_factories = int(f.readline())
        for i in range(n_factories):
            arr = f.readline().split()
            arr = list(map(int, arr))
            dist.append(arr)
        
        f.readline()
        for i in range(n_factories):
            arr = f.readline().split()
            arr = list(map(int, arr))
            flow.append(arr)
        
        best_sol, best_sol_value, dt = ils(n_factories, dist, flow, inverse_k_perturbator, 10)
        data.append([file[:-4] , best_sol_value, dt])
        print(f'Best solution: {best_sol}')
        print(f'Best solution flow value: {best_sol_value}')
        print(f'Time: {dt}\n')
        with open(f'ils_outputs/{file}', 'w') as output_file:
            print(*best_sol, file=output_file)

Best solution: [14, 3, 17, 19, 4, 16, 10, 18, 13, 6, 15, 8, 12, 5, 7, 0, 2, 9, 1, 11]
Best solution flow value: 721242
Time: 0.10699868202209473



Best solution: [23, 33, 4, 10, 15, 37, 8, 13, 31, 24, 3, 26, 25, 22, 20, 27, 21, 17, 6, 7, 28, 36, 30, 1, 12, 16, 35, 19, 2, 39, 14, 38, 32, 5, 0, 18, 11, 34, 9, 29]
Best solution flow value: 3233552
Time: 1.008500099182129

Best solution: [17, 10, 25, 3, 27, 32, 56, 55, 23, 11, 21, 24, 52, 36, 59, 42, 44, 57, 18, 43, 6, 47, 48, 0, 16, 49, 50, 26, 20, 46, 15, 31, 8, 33, 2, 39, 13, 29, 41, 28, 40, 7, 1, 38, 9, 30, 4, 22, 35, 19, 45, 51, 37, 53, 12, 54, 5, 14, 58, 34]
Best solution flow value: 7421824
Time: 3.8709959983825684

Best solution: [69, 35, 20, 48, 42, 9, 21, 29, 6, 13, 45, 40, 36, 47, 14, 55, 74, 31, 50, 79, 12, 44, 49, 17, 77, 78, 70, 22, 54, 59, 24, 4, 16, 75, 71, 38, 57, 25, 0, 10, 27, 34, 7, 73, 65, 1, 72, 19, 8, 30, 28, 37, 41, 53, 62, 39, 52, 5, 76, 23, 3, 61, 67, 11, 64, 32, 33, 15, 46, 66, 18, 26, 51, 63, 68, 60, 58, 56, 2, 43]
Best solution flow value: 13986514
Time: 7.61600136756897

Best solution: [79, 90, 24, 80, 86, 13, 0, 84, 55, 8, 36, 17, 20, 72, 71, 46, 88, 68

In [55]:
ils_df = pd.DataFrame(data, columns=['sample', 'ILS best value', 'ILS time'])
df = ls_df.set_index('sample').join(ils_df.set_index('sample'))

In [56]:
df

,LS best value,LS time,ILS best value,ILS time
sample,,,,
tai20a,736450,0.011997,721242,0.106999
tai40a,3260316,0.154501,3233552,1.008500
tai60a,7467702,0.594502,7421824,3.870996
tai80a,14061060,1.082001,13986514,7.616001
tai100a,21771672,1.810001,21704750,15.742002
